<h1 align='center'>Twitter API and NLP</h1>

<h2>Importación de las Librerías</h2>

In [1]:
import ast
import copy
from datetime import datetime, timedelta
from dateutil import parser
import json
import numpy as np
import os
import pandas as pd
import pytz
import re
import requests
import time
from tqdm import tqdm
import tweepy

from utils import functions, querys
print('All libraries successfully loaded...')

import urllib3 # Just for the runner and debugger
import importlib # Delete for production and use importlib.reload(<module>)

All libraries successfully loaded...


<h2>Debugging Toolkit</h2>

In [2]:
import sys

# Debugging message for first loadings
environ = sys.prefix[sys.prefix.rindex('\\')+1:]
print('Si falló el último paso, abra Anaconda Prompt e ingrese los siguientes comandos:\n\n' +
      f'1) conda activate {environ}\n' + f'2) pip install -r {os.getcwd()}\\requirements.txt')

# Typing library is not built-in
from typing import Optional

# And this debugging controller requires it to work
def display_control(df: pd.DataFrame, value: Optional[int] = None,
                    axis: Optional[int] = 0):
    
    dict_axis = {0: "rows", 1: "columns"}
    
    if value:
        pd.set_option(f"max_{dict_axis[axis]}", value)
        
    display(df)
    pd.reset_option(f"max_{dict_axis[axis]}")
    
# Timer start
start = time.time() 
    
# Pandas configuration for tqdm
tqdm.pandas()

Si falló el último paso, abra Anaconda Prompt e ingrese los siguientes comandos:

1) conda activate Twitter-API
2) pip install -r c:\Users\nicol\Proyectos\GitHub\Consultorías\Twitter\trade_union_tweets\requirements.txt


<h2>Configuraciones Generales</h2>

In [3]:
# We store our starting day and start the time
tz = pytz.timezone('America/Santiago')
santiago_now = datetime.now(tz)

# We define a day range to store
stored_since = santiago_now - timedelta(days=7)
date_stored = stored_since.strftime('%Y%m%d')

# Creating IDs for storage and starting our time counter 
date_id = santiago_now.strftime("%Y%m%d")
time_id = santiago_now.strftime("%H%M")

# dict_accounts = {"cutchile": 289376883, "confed_bancaria": 607178284,
#                  "confusamchile": 203352616, "fenpruss": 75044282,
#                  "cotraporchi": 3402211569, "confecobre": 262411747}

dict_accounts = {"fenpruss": 75044282, "fenassap": 750476199810457601,
                 "confusamchile": 203352616} #, "fenpruss": 75044282,
#                  "cotraporchi": 3402211569, "confecobre": 262411747}

# Limit date definition
since = "2017-01-01"
dttm_since = f"{since}T00:00:00Z"

# Pandas configuration
tqdm.pandas()

print(f"Descargando datos de {len(dict_accounts.keys())} cuentas, desde el "
      f"{since} hasta hoy...")

Descargando datos de 3 cuentas, desde el 2017-01-01 hasta hoy...


<h2>Obtención de las Credenciales</h2>

In [4]:
# We obtain the script's path independent of our OS
path_children = ['credentials', 'credentials.txt']
credentials_path = functions.script_path(path_children)

# We instantiate and fill our dictionary
with open(credentials_path) as file:
    lines = file.readlines()
    credentials = dict()
    
    for line in lines:
        credentials.update(ast.literal_eval(line))
        
print(f"Credenciales cargadas: {', '.join([*credentials.keys()])}...")

Credenciales cargadas: twitter_key, twitter_secret, twitter_token, token_access, token_secret...


<h2>Levantamiento de los Clientes</h2>

In [5]:
# We obtain and set our token for Twitter
os.environ["BEARER_TOKEN"] = credentials['twitter_token']
bearer_token = os.environ.get("BEARER_TOKEN")

print("Variables ambientales levantadas, preparando funciones clave...")

Variables ambientales levantadas, preparando funciones clave...


<h2>Definición de Funciones Clave</h2>

In [6]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", search_url, auth=bearer_oauth, params=params)
#     print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

print("Funciones definidas y listas para usarse. Extrayendo tweets...")

Funciones definidas y listas para usarse. Extrayendo tweets...


<h2>Loop de Extracción de Datos</h2>

In [8]:
# Preparing our complete information dictionary
tweets_by_account = {account: [] for account in dict_accounts.keys()}

for account, account_id in dict_accounts.items():
    while True:
        search_url = "https://api.twitter.com/2/tweets/search/all"
        query_params = {'query': f"to:{account_id} -is:retweet",
                        'tweet.fields': 'created_at,public_metrics,entities',
                        'start_time': dttm_since,
                        "max_results": 500}

        if len(tweets_by_account[account]) > 0:
            meta = json_response['meta']

            if 'next_token' in meta:
                query_params['next_token'] = meta['next_token']

            else:
                break

        json_response = connect_to_endpoint(search_url, query_params)
        data, meta = [json_response[key] for key in json_response.keys()]
        
        tweets_by_account[account].append({'meta': meta, 'data': data})

        print(f"Se extrajeron {len(json_response['data'])} tweets de "
              f"{account.upper()}, donde la última publicación fue el día "
              f"{parser.parse(json_response['data'][-1]['created_at']).date()}...")

print('Información descargada exitosamente...')

Se extrajeron 480 tweets de FENPRUSS, donde la última publicación fue el día 2021-03-12...
Se extrajeron 494 tweets de FENPRUSS, donde la última publicación fue el día 2020-05-28...
Se extrajeron 496 tweets de FENPRUSS, donde la última publicación fue el día 2019-03-30...
Se extrajeron 150 tweets de FENPRUSS, donde la última publicación fue el día 2017-01-07...
Se extrajeron 382 tweets de FENASSAP, donde la última publicación fue el día 2017-03-28...
Se extrajeron 483 tweets de CONFUSAMCHILE, donde la última publicación fue el día 2021-03-26...
Se extrajeron 484 tweets de CONFUSAMCHILE, donde la última publicación fue el día 2020-10-14...
Se extrajeron 496 tweets de CONFUSAMCHILE, donde la última publicación fue el día 2020-05-08...
Se extrajeron 500 tweets de CONFUSAMCHILE, donde la última publicación fue el día 2019-10-22...
Se extrajeron 498 tweets de CONFUSAMCHILE, donde la última publicación fue el día 2017-03-26...
Se extrajeron 29 tweets de CONFUSAMCHILE, donde la última publica

<h2>Generación del Dataframe y Descarga como Excel</h2>

In [10]:
elem

{'text': '@ConfusamChile https://t.co/iP3sD1f5Ok',
 'created_at': '2017-01-16T23:09:41.000Z',
 'public_metrics': {'retweet_count': 2,
  'reply_count': 1,
  'like_count': 0,
  'quote_count': 0},
 'entities': {'mentions': [{'start': 0,
    'end': 14,
    'username': 'ConfusamChile',
    'id': '203352616'}],
  'urls': [{'start': 15,
    'end': 38,
    'url': 'https://t.co/iP3sD1f5Ok',
    'expanded_url': 'https://twitter.com/mileschile/status/821019533443993600',
    'display_url': 'twitter.com/mileschile/sta…'}]},
 'id': '821132343951978497',
 'url': 'https://t.co/iP3sD1f5Ok'}

In [11]:
importlib.reload(functions)
list_dfs = []

aux = 1

# Generating list of data and storage
for account in tqdm(dict_accounts.keys()):
    raw_list = [elem['data'] for elem in tweets_by_account[account]]
    flat_list = [item for sublist in raw_list for item in sublist]
    
    if len(flat_list) == 0:
        continue
    
    # Enritching
    list_local = []
    
    for elem in flat_list:
        print(elem)
        aux += 1

        if aux > 12:
            break
        
        try: # Sometimes there is no url to be found
            elem['url'] = elem['entities']['urls'][0]['url']
        except Exception as e:
            elem['url'] = f"Error: {e}"
            
        dict_local = {**elem, **elem['public_metrics']}
        list_local.append(dict_local)
        
    df_local = pd.DataFrame(list_local)
    df_local['by'] = account
    
    # Fixing date columns
    df_local['created_at'] = pd.to_datetime(df_local['created_at'],
                                            infer_datetime_format=True)
    
    df_local['created_at'] = df_local['created_at'].dt.strftime('%F %T')
    
    # Selecting columns to maintain
    list_dfs.append(df_local[['created_at', 'by', 'text','retweet_count',
                              'reply_count', 'like_count', 'quote_count', 'url']])
    
    # Writing our df to an Excel file
    path_children = ['storage', f"{date_id}_{time_id}_trade_union_tweets.xlsx"]
    filepath = str(functions.script_path(path_children))
    functions.write_excel(filepath, account, list_dfs[-1], errase='Sheet1')

# Generating concatenated DataFrame
df = pd.concat(list_dfs, ignore_index=True)
print('Tabla correctamente descargada y disponible para su análisis...')
display(df.tail(5))

  0%|          | 0/3 [00:00<?, ?it/s]

{'entities': {'urls': [{'start': 149, 'end': 172, 'url': 'https://t.co/nV54PaB61E', 'expanded_url': 'https://twitter.com/ingralej/status/1532075533231476736/photo/1', 'display_url': 'pic.twitter.com/nV54PaB61E', 'media_key': '3_1532075528928337920'}], 'mentions': [{'start': 0, 'end': 9, 'username': 'Fenpruss', 'id': '75044282'}, {'start': 10, 'end': 23, 'username': 'gabrielboric', 'id': '73981088'}]}, 'created_at': '2022-06-01T19:04:06.000Z', 'id': '1532075533231476736', 'public_metrics': {'retweet_count': 2, 'reply_count': 0, 'like_count': 2, 'quote_count': 0}, 'text': '@Fenpruss @gabrielboric Ya está garantizada en la Constitución actual y de hecho hay patologías que tienen especial atención. La NC no lo garantiza. https://t.co/nV54PaB61E', 'url': 'https://t.co/nV54PaB61E'}
{'entities': {'hashtags': [{'start': 106, 'end': 121, 'tag': 'mentirapublica'}, {'start': 122, 'end': 138, 'tag': 'cuentapublica22'}], 'mentions': [{'start': 0, 'end': 9, 'username': 'Fenpruss', 'id': '75044282'},

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\nicol\\Proyectos\\GitHub\\Consultorías\\Twitter\\trade_union_tweets\\storage\\20220601_1934_trade_union_tweets.xlsx'